In [1]:
import pandas as pd
import numpy as np

import nltk
from nltk import word_tokenize

from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.decomposition import NMF
import nltk


import glob
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import string
df = pd.read_csv('bitcoin_twitter.csv')
df

,id,created_at,favorite_count,retweet_count,text
0,1427917464885682177,2021-08-18 08:57:07,10,1,@CypherPaints @RD_btc where's the dip? https:/...
1,1427776452053901318,2021-08-17 23:36:47,0,0,@mutatrum @SGBarbour @Endorsen @NickSzabo4 @ha...
2,1427759906170548227,2021-08-17 22:31:02,3,0,@sovereignhodler @paoloardoino @Liquid_BTC @bi...
3,1427756195390136323,2021-08-17 22:16:18,10,1,@Esky33junglist @Mandrik @TheGuySwann confiden...
4,1427755868171509761,2021-08-17 22:15:00,13,1,@paoloardoino i think for longer-term cold sto...
...,...,...,...,...,...
29864,1361157185946836995,2021-02-15 03:35:36,0,0,@AriDavidPaul @Readwiseio save thread
29865,1360861621350932482,2021-02-14 08:01:08,0,0,@SahilBloom @Readwiseio save thread
29866,1360479655199838210,2021-02-13 06:43:20,0,0,@yassineARK @Readwiseio save thread
29867,1358997025656696836,2021-02-09 04:31:54,0,0,@markoff H


In [2]:
import re
import string

alphanumeric = lambda x: re.sub('\w*\d\w*', ' ', x)
punc_lower = lambda x: re.sub('[%s]' % re.escape(string.punctuation), ' ', x.lower())
rid_handles = lambda x: re.sub('@\S+', ' ', x)
links = lambda x: re.sub('https?:\/\/\S+', ' ', x)
breaks = lambda x: re.sub('\n', ' ', x)
df['text'] = df['text'].map(rid_handles).map(links).map(breaks).map(alphanumeric).map(punc_lower)

In [3]:
df['text'] = df['text'].astype('string')

In [4]:
mask = df['created_at'] >= '2021-05'
df[mask]

,id,created_at,favorite_count,retweet_count,text
0,1427917464885682177,2021-08-18 08:57:07,10,1,where s the dip
1,1427776452053901318,2021-08-17 23:36:47,0,0,i think people say that about people...
2,1427759906170548227,2021-08-17 22:31:02,3,0,ct is confidential to everyone except ...
3,1427756195390136323,2021-08-17 22:16:18,10,1,confidential transactions
4,1427755868171509761,2021-08-17 22:15:00,13,1,i think for longer term cold storage people ...
...,...,...,...,...,...
29796,1390026999792427009,2021-05-05 19:33:56,4,0,assumed that part was obvious
29797,1390025425124958210,2021-05-05 19:27:41,119,2,how it started
29798,1390004608739647488,2021-05-05 18:04:58,2,0,save thread
29799,1389696368239775744,2021-05-04 21:40:08,0,0,save thread


In [5]:
df['text']

0                                       where s the dip   
1                  i think people say that about people...
2                ct is confidential to everyone except ...
3                                confidential transactions
4          i think for longer term cold storage people ...
                               ...                        
29864                                          save thread
29865                                          save thread
29866                                          save thread
29867                                                    h
29868                                                     
Name: text, Length: 29869, dtype: string

In [6]:

from sklearn.feature_extraction import text

In [7]:
df = df[mask]
df

,id,created_at,favorite_count,retweet_count,text
0,1427917464885682177,2021-08-18 08:57:07,10,1,where s the dip
1,1427776452053901318,2021-08-17 23:36:47,0,0,i think people say that about people...
2,1427759906170548227,2021-08-17 22:31:02,3,0,ct is confidential to everyone except ...
3,1427756195390136323,2021-08-17 22:16:18,10,1,confidential transactions
4,1427755868171509761,2021-08-17 22:15:00,13,1,i think for longer term cold storage people ...
...,...,...,...,...,...
29796,1390026999792427009,2021-05-05 19:33:56,4,0,assumed that part was obvious
29797,1390025425124958210,2021-05-05 19:27:41,119,2,how it started
29798,1390004608739647488,2021-05-05 18:04:58,2,0,save thread
29799,1389696368239775744,2021-05-04 21:40:08,0,0,save thread


In [ ]:
df.info()

In [ ]:
def preprocess_text(text):
    tokens = word_tokenize(text)
    tokens = [re.sub('[%s]' % re.escape(string.punctuation), '', token) for token in tokens]
    lemmatiser = WordNetLemmatizer()
    lemmas = [lemmatiser.lemmatize(token) for token in tokens]
    return lemmas

In [ ]:
stops = text.ENGLISH_STOP_WORDS.union(['don'])

In [ ]:
#Tried Lemmatizing and Stemming, Lemmatizing seemed to give generally more understandable tokens
#though lemmatizing did not decrease corpus size significantly 
def preprocess_text(text):
    """preprocessing text, tokenizing words that are strings longer than 2 letters
    lemmatising tokens, adding new stopwords to stopwords list and removing all stopwords"""
    from nltk.corpus import stopwords
    #Tokenize words while ignoring punctuation
    #letters only, strings longer than 2 letters
    tokeniser = RegexpTokenizer(r'[A-Za-z]{3,}')
    tokens = tokeniser.tokenize(text)
     # lemmatise 
    lemmatiser = WordNetLemmatizer()
    lemmas = [lemmatiser.lemmatize(token,pos='v') for token in tokens]
    #ps=PorterStemmer()
    #ports=[ps.stem(token.lower())for token in tokens]
    
    # Remove creating and removing stopwords
    stopwords= stopwords.words('english')
    new_stopwords=['don'] 
    stopwords.extend(new_stopwords)
    keywords= [lemma for lemma in lemmas if lemma not in stopwords]
    #keywords= [port for port in ports if port not in stopwords]
    return keywords

In [8]:
docs = df['text']

In [ ]:
import nltk

In [9]:
def make_topics(docs, preprocessor, vectorizer, topic_modeler, print_n_words=15):
    """A very simple pipeline."""
    
    # Apply preprocessor, vectorizer, and topic modeler.
    if preprocessor is not None:
        docs = docs.apply(preprocessor)
    
    # Vectorize documents into a document-word matrix.
    doc_word_vectors = vectorizer.fit_transform(docs)
    
    # Fit the topic model.
    doc_topic_vectors = topic_modeler.fit_transform(doc_word_vectors)
    
    # Print the topics.
    vocab = vectorizer.get_feature_names()
    for idx, topic in enumerate(topic_modeler.components_):
        # Select the top 15 words in vocab for this topic.
        top_words = [vocab[i].upper() for i in topic.argsort()[:-print_n_words-1:-1]]
        print(f"Topic {idx}:\n", ", ".join(top_words), "\n")
    
    return doc_topic_vectors

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF

docs = df['text']
preprocessor = None
vectorizer = TfidfVectorizer(stop_words= 'english', min_df=5)
topic_modeler = NMF(5, random_state=10, max_iter = 1000)  #random state to make the results deterministic

make_topics(docs, preprocessor, vectorizer, topic_modeler);

Topic 0:
 BITCOIN, FIXES, MONEY, BUY, WORLD, CRYPTO, TIME, FUTURE, ENERGY, YEARS, DIGITAL, FIAT, MINING, STANDARD, GOLD 

Topic 1:
 YES, EXACTLY, HELL, THANK, DID, AH, FUCK, ANSWER, PROBABLY, SIR, OK, READING, FREE, AGO, REALLY 

Topic 2:
 DON, LIKE, JUST, PEOPLE, KNOW, THINK, NEED, TIME, WANT, RIGHT, THANKS, MONEY, MAKE, WORK, LL 

Topic 3:
 LOL, DID, NICE, LOVE, OH, GREAT, SHIT, EXACTLY, THREAD, FAIR, USE, LITERALLY, TALK, REKT, TWEETS 

Topic 4:
 GOOD, MORNING, LUCK, POINT, MAN, LOOKING, IDEA, REALLY, DAY, THING, BAD, TIMES, TODAY, QUESTION, TIME 



/opt/anaconda3/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:312: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(("The 'init' value, when 'init=None' and "


In [11]:
from sklearn.feature_extraction.text import CountVectorizer

In [12]:
vectorizer = CountVectorizer(stop_words='english', min_df=5)
doc_word = vectorizer.fit_transform(docs)
doc_word.shape

(28322, 6099)

In [13]:
doc_word

<28322x6099 sparse matrix of type '<class 'numpy.int64'>'
	with 173433 stored elements in Compressed Sparse Row format>

In [14]:
dff = pd.DataFrame(doc_word.toarray(), index=docs, columns=vectorizer.get_feature_names())
dff

,abandon,abandoned,aber,ability,abject,able,abolish,abroad,absence,absolute,...,zealand,zen,zero,zerobasefee,zeus,zombie,zone,zonte,zoom,zu
text,,,,,,,,,,,,,,,,,,,,,
where s the dip,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
i think people say that about people who co founded the company we invented it a company being an organization of individuals to common purpose hyperbitcoinization,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ct is confidential to everyone except sender and recipient as value and type are encrypted federation block signers have no visibility just like your full node only sender and recipient can decrypt anyone can verify values add up due to use of additive homomorphic crypto amp zkp,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
confidential transactions,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
i think for longer term cold storage people would want storage also adoption looking good just need more exchanges following lead in deploying and reinvesting in bitcoin for more cross exchange transfer,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
assumed that part was obvious,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
how it started,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
save thread,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
stopWords = {'of', 'more', 'others', 'and', 'amoungst', 'ours', 'forty', 'elsewhere', 'own', 'anyone', 'but', 'becoming', 'fill', 'herself', 'both', 'their', 'will', 'all', 'less', 'two', 'many', 'four', 'eleven', 'them', 'towards', 'latterly', 'might', 'alone', 'himself', 'there', 'beforehand', 'three', 'again', 'must', 'whenever', 'move', 'across', 'empty', 'this', 'everywhere', 'seeming', 'those', 'therein', 'whole', 'these', 'yourself', 'while', 'to', 'co', 'me', 'fifty', 'for', 'my', 'its', 'still', 'sometimes', 'from', 'she', 'sincere', 'could', 'done', 'most', 'above', 'indeed', 'mostly', 'several', 'among', 'anyway', 'perhaps', 'only', 'twelve', 'give', 'between', 'after', 'an', 'when', 'you', 'into', 'put', 'meanwhile', 'over', 'our', 'well', 'during', 'also', 'other', 'nor', 'been', 'were', 'hundred', 'throughout', 'somehow', 'already', 'become', 'each', 'call', 'hereby', 'which', 'thereby', 'de', 'eg', 'full', 're', 'whereafter', 'on', 'they', 'sometime', 'namely', 'inc', 'detail', 'although', 'go', 'through', 'ltd', 'us', 'latter', 'against', 'her', 'everyone', 'then', 'so', 'can', 'any', 'see', 'ie', 'hers', 'anyhow', 'top', 'neither', 'hereafter', 'is', 'per', 'thereafter', 'how', 'a', 'get', 'be', 'yours', 'wherein', 'side', 'seemed', 'third', 'very', 'another', 'few', 'we', 'upon', 'whatever', 'sixty', 'would', 'beyond', 'con', 'math', 'same', 'had', 'ever', 'nothing', 'else', 'whether', 'becomes', 'off', 'enough', 'toward', 'one', 'afterwards', 'bill', 'i', 'next', 'thru', 'such', 'seem', 'whoever', 'together', 'hereupon', 'what', 'find', 'his', 'every', 'do', 'serious', 'everything', 'cannot', 'cant', 'under', 'without', 'eight', 'themselves', 'because', 'via', 'show', 'here', 'than', 'however', 'along', 'please', 'thus', 'twenty', 'herein', 'either', 'whither', 'nobody', 'with', 'became', 'who', 'may', 'keep', 'in', 'he', 'below', 'something', 'much', 'except', 'beside', 'whence', 'interest', 'whose', 'due', 'mill', 'behind', 'fifteen', 'otherwise', 'almost', 'nine', 'system', 'couldnt', 'back', 'rather', 'your', 'anything', 'front', 'nevertheless', 'seems', 'take', 'now', 'last', 'hasnt', 'myself', 'thence', 'about', 'wherever', 'somewhere', 'have', 'out', 'until', 'therefore', 'hence', 'yet', 'by', 'within', 'up', 'thick', 'where', 'am', 'further', 'five', 'should', 'least', 'since', 'ten', 'cry', 'around', 'none', 'always', 'anywhere', 'six', 'never', 'describe', 'him', 'has', 'once', 'not', 'onto', 'un', 'it', 'fire', 'before', 'that', 'often', 'amongst', 'being', 'someone', 'etc', 'if', 'some', 'no', 'whom', 'found', 'are', 'whereby', 'former', 'first', 'moreover', 'was', 'made', 'nowhere', 'as', 'mine', 'noone', 'though', 'or', 'yourselves', 'formerly', 'bottom', 'ourselves', 'itself', 'name', 'amount', 'part', 'down', 'the', 'whereas', 'thereupon', 'whereupon', 'besides', 'too', 'even', 'at', 'thin', 'why'}

In [16]:
def get_top_n_words(corpus, n=None):
   
    
    vec = CountVectorizer().fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in     vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

In [17]:
tops = get_top_n_words(docs, n=100)

In [18]:
new_tops = []
for i in range(100):
    if tops[i][0] not in stopWords:
        new_tops.append(tops[i])
    
    

In [19]:
new_tops

[('bitcoin', 4170),
 ('just', 1628),
 ('people', 1512),
 ('like', 1456),
 ('don', 1423),
 ('think', 903),
 ('good', 883),
 ('time', 866),
 ('money', 828),
 ('need', 735),
 ('know', 730),
 ('amp', 646),
 ('way', 581),
 ('want', 569),
 ('make', 551),
 ('right', 543),
 ('new', 520),
 ('yes', 518),
 ('going', 513)]

In [ ]:
def display_topics(model, feature_names, no_top_words, topic_names=None):
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [ ]:
nmf_model = NMF(5)
doc_topic = nmf_model.fit_transform(doc_word)

In [ ]:
topic_word = pd.DataFrame(nmf_model.components_.round(5),
             index = ["component_1","component_2","component_3","component_4","component_5"],
             columns = vectorizer.get_feature_names())
topic_word

In [ ]:
display_topics(nmf_model, vectorizer.get_feature_names(), 10)

In [ ]:
H = pd.DataFrame(doc_topic.round(5), index=docs,
                
             columns = ["component_1","component_2","component_3","component_4","component_5"])
H

In [ ]:
H.sort_values('component_3', ascending=False)

In [ ]:
H.sample(25)

In [ ]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud

vocab = vectorizer.get_feature_names()

# Generate a word cloud image for given topic
def draw_word_cloud(index):
    
    imp_words_topic = ""
    components = nmf_model.components_[index]
    vocab_comp = zip(vocab, components)
    sorted_words = sorted(vocab_comp, key=lambda x:x[1], reverse=True)[:40]
    
    for word in sorted_words:
        imp_words_topic = imp_words_topic + " " + word[0]

    wordcloud = WordCloud(width=600, height=400).generate(imp_words_topic)
    plt.figure(figsize=(5,5))
    plt.imshow(wordcloud)
    plt.axis("off")
    plt.tight_layout()
    plt.show()
 
# draw all topics to compare
for i in range(5):
    draw_word_cloud(i)

In [ ]:
import sys
print(sys.executable)

In [ ]:
/anaconda3/bin/python -m pip install wordcloud